# A signed list of transactions

Following the slides we can do the same task with a list of transactions

```
Tim pays Joel $3
Joel pays Kevin $1.5
Tim pays Joel $3
```


In [1]:
from hashlib import sha256

def simple_hash_func(value):
    return sha256('{}'.format(value).encode()).hexdigest()[-8:]

In [2]:
transactions = [
    ['2017-12-12', 'Tim pays Joel $3'],
    ['2017-12-13', 'Joel pays Kevin $1.5'],
    ['2017-12-13', 'Tim pays Joel $3']
]

We will link this set of transaction to each other by computing a hash for each one that includes the has of the previous transaction. This way the third transaction is linked to the second, which is linked to the first.

In [3]:
def sign_transactions(transactions):
    signed = []
    last_signature = ''

    for date, content in transactions:
        last_signature = simple_hash_func(date + content + last_signature)
        signed.append([date, content, last_signature])
        
    return signed

signed_transactions = sign_transactions(transactions)

for date, content, signature in signed_transactions:
    print('\t'.join([date, content, signature]))

2017-12-12	Tim pays Joel $3	5bb3d468
2017-12-13	Joel pays Kevin $1.5	33417c49
2017-12-13	Tim pays Joel $3	697bb016


We can now try and manipulate an earlier transaction in our chain and see what happens.

We make a fairly simple modification, swapping a `5` and a `.` in the second transaction. As a result Kevin wil lreceive $15 instead of $1.5. A third party will be able to tell that this modification has happened because the signature for the last transaction differs from what they have.

In [4]:
transactions[1][1] = 'Joel pays Kevin $15.'

new_signed_transactions = sign_transactions(transactions)

for (date, content, new_signature), (_, _, old_signature) in zip(new_signed_transactions, signed_transactions):
    print('\t'.join([date, content]))
    print('\t\t\tnew: ', new_signature)
    print('\t\t\told: ', old_signature)

2017-12-12	Tim pays Joel $3
			new:  5bb3d468
			old:  5bb3d468
2017-12-13	Joel pays Kevin $15.
			new:  e4169389
			old:  33417c49
2017-12-13	Tim pays Joel $3
			new:  ed1d0cbb
			old:  697bb016


Note how the value of the hash function for the second transation is completely different from its old value even though we just swapped a `5` and `.`. Because the hash value of the previous transaction feeds into the current transaction's hash value we can tell by looking just at the last transaction that something has gone wrong or was manipulated along the way. We do not have to go through every single transaction to be able to tell.

The problem with using a simple hash function like this is that it is not very expensive to  create a fake blockchain where you recompute all the hashes yourself.